<a href="https://colab.research.google.com/github/Adammalik28/UAS-MachineLearning/blob/master/Laporan%20Prediksi%20Koding%20(Algoritma%20Naive%20Bayes).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laporan Prediksi Koding (Algoritma Naive Bayes)

# import module

In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, KBinsDiscretizer
# KITA GANTI MODELNYA KE CATEGORICALNB (MASIH KELUARGA NATIVE BAYES)
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Memuat Data

In [28]:
# --- 1. Memuat Data ---
try:
    df = pd.read_csv('DP.Proyek1 - Sheet 1.csv')
except FileNotFoundError:
    print("File dataset tidak ditemukan.")
    exit()

# Pembersihan & Persiapan Data

In [29]:
# --- 2. Pembersihan & Persiapan Data ---
# Mengatasi 'Jumlah Investasi'
df['Investasi_Angka'] = pd.to_numeric(df['Jumlah Investasi'].astype(str).str.replace(',', ''), errors='coerce')
# Mengatasi 'TKI'
df['TKI_Angka'] = pd.to_numeric(df['TKI'], errors='coerce')

# Mengisi data yang hilang (NaN)
df['Investasi_Angka'] = df['Investasi_Angka'].fillna(0)
df['TKI_Angka'] = df['TKI_Angka'].fillna(0)

# Membuang baris di mana target atau fitur utama kosong
df = df.dropna(subset=['Uraian Risiko Proyek', 'kecamatan_usaha'])

# Feature Engineering (SANGAT PENTING)

In [30]:
# --- 3. Feature Engineering (SANGAT PENTING) ---

# A. Binning (Discretization) Fitur Numerik
# Kita ubah angka kontinu (Investasi, TKI) menjadi 5 kategori (bins)
# 'quantile' berarti setiap bin akan punya jumlah data yang kurang lebih sama
# 'ordinal' berarti outputnya adalah angka 0, 1, 2, 3, 4
kbins_investasi = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')
kbins_tki = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')

# Hati-hati dengan data duplikat di quantile, tambahkan sedikit noise jika error
try:
    df['Investasi_Binned'] = kbins_investasi.fit_transform(df[['Investasi_Angka']])
except ValueError:
    # Jika ada error krn data duplikat, tambahkan noise kecil
    noise = np.random.normal(0, 1e-6, df['Investasi_Angka'].shape)
    df['Investasi_Binned'] = kbins_investasi.fit_transform(df[['Investasi_Angka']] + noise)

try:
    df['TKI_Binned'] = kbins_tki.fit_transform(df[['TKI_Angka']])
except ValueError:
    noise = np.random.normal(0, 1e-6, df['TKI_Angka'].shape)
    df['TKI_Binned'] = kbins_tki.fit_transform(df[['TKI_Angka']] + noise)


# B. Encoding Fitur Kategorikal (Kecamatan)
# Karena kita pakai CategoricalNB, LabelEncoder sudah benar
le_kecamatan = LabelEncoder()
df['Kecamatan_Encoded'] = le_kecamatan.fit_transform(df['kecamatan_usaha'])

# C. Encoding Target (Risiko)
le_risiko = LabelEncoder()
df['Risiko_Encoded'] = le_risiko.fit_transform(df['Uraian Risiko Proyek'])

/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_discretization.py:306: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(


# Memisahkan Fitur (X) dan Target (y)

In [31]:
# --- 4. Memisahkan Fitur (X) dan Target (y) ---
# Fitur kita SEKARANG SEMUA KATEGORIKAL
fitur = ['Kecamatan_Encoded', 'Investasi_Binned', 'TKI_Binned']
target = 'Risiko_Encoded'

X = df[fitur]
y = df[target]

# Membagi Data Latih dan Data Uji

In [32]:
# --- 5. Membagi Data Latih dan Data Uji ---
# 'stratify=y' sangat penting untuk data tidak seimbang
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Melatih Model CATEGORICAL NAIVE BAYES

In [33]:
# --- 6. Melatih Model CATEGORICAL NAIVE BAYES ---
# Ini adalah model yang tepat untuk data kita
model_nb = CategoricalNB()
model_nb.fit(X_train, y_train)

CategoricalNB()

# Membuat Prediksi

In [34]:
y_pred = model_nb.predict(X_test)

# Laporan Hasil

In [35]:
# --- 8. Laporan Hasil ---
print("\n--- LAPORAN HASIL PREDIKSI (CATEGORICAL NAIVE BAYES) ---")

# Laporan Akurasi
accuracy = accuracy_score(y_test, y_pred)
print(f"Akurasi Model: {accuracy * 100:.2f}%")

# Laporan Prediksi (Laporan Klasifikasi)
print("\nLaporan Klasifikasi (Prediksi):")
target_names = le_risiko.classes_
# zero_division=0 untuk menghindari error jika ada kelas yg tidak terprediksi
print(classification_report(y_test, y_pred, target_names=target_names, zero_division=0))


--- LAPORAN HASIL PREDIKSI (CATEGORICAL NAIVE BAYES) ---
Akurasi Model: 78.33%

Laporan Klasifikasi (Prediksi):
                 precision    recall  f1-score   support

Menengah Rendah       0.85      0.37      0.52      1003
Menengah Tinggi       0.00      0.00      0.00       303
         Rendah       0.78      0.98      0.87      3831
         Tinggi       1.00      0.01      0.01       156

       accuracy                           0.78      5293
      macro avg       0.66      0.34      0.35      5293
   weighted avg       0.75      0.78      0.73      5293

